<a href="https://colab.research.google.com/github/julianavmgiusti/soulcode/blob/jvmg-soulcode/Atividade_mongo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.3 MB/s eta 0:00:00


In [ ]:
pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 KB 4.1 MB/s eta 0:00:00


In [ ]:
import pymongo
import pandas as pd
import numpy as np
from pymongo import MongoClient
import pandera as pa

In [ ]:
import os
from google.cloud import storage

In [ ]:
pd.set_option('display.max_columns',100)

##Carregamento Cloud Storage 

In [ ]:
serviceAccount = '/content/crafty-dynamics-377819-c4ea9df48734.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#Conector MongoDB Atlas
uri = "mongodb+srv://aula0.jhjxpzb.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,tls=True,tlsCertificateKeyFile='/content/X509-cert-1772805837742033332.pem')

In [ ]:
client = storage.Client() #opcional
bucket = client.get_bucket('julianagiusti01') #opcional
bucket.blob('ocorrenciass.csv') #opcional
path_cenipa = 'gs://julianagiusti01/brutos/ocorrenciass.csv' #obrigatorio
path_airbnb = 'gs://julianagiusti01/brutos/amsterdam_weekdays.csv'
path_alespe = 'gs://julianagiusti01/brutos/depsp.csv'

##EXtração de dados brutos do Google Cloud


In [ ]:
df_cenipa = pd.read_csv(path_cenipa, sep=';', encoding='ISO-8859-1')

In [ ]:
df_airbnb = pd.read_csv(path_airbnb)

In [ ]:
df_alespe = pd.read_csv(path_alespe)

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
del client

##Criando Collections e Enviando Brutos para o MongoDB

In [ ]:
db = client['Brutos']
collection_cenipa = db['cenipa']
collection_airbnb = db['airbnb']
collection_alespe = db['alespe']

In [ ]:
#Evio dos dados brutos das coleções (cenipa, airbnb e alespe) para o MongoDB
df_dict_cenipa = df_cenipa.to_dict("records")
df_dict_airbnb = df_airbnb.to_dict("records")
df_dict_alespe = df_alespe.to_dict("recods")

collection_cenipa.insert_many(df_dict_cenipa)
collection_airbnb.insert_many(df_dict_airbnb)
collection_alespe.insert_many(df_dict_alespe)

<ipython-input-32-01e4d8c81f62>:4: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  df_dict_alespe = df_alespe.to_dict("recods")


In [ ]:
doc_count_cenipa = collection_cenipa.count_documents({})
print(doc_count_cenipa)

doc_count_airbnb = collection_airbnb.count_documents({})
print(doc_count_airbnb)

doc_count_alespe = collection_alespe.count_documents({})
print(doc_count_alespe)

6769
1103
542395


In [ ]:
df_dict_cenipa = df_cenipa.to_dict("records")

collection_cenipa.insert_many(df_dict_cenipa)

In [ ]:
doc_count_cenipa = collection_cenipa.count_documents({})
print(doc_count_cenipa)

6769


##Transformação (Cenipa)

In [ ]:
df_cenipa.drop(['codigo_ocorrencia1',
         'codigo_ocorrencia2',
         'codigo_ocorrencia3',
         'codigo_ocorrencia4',
         'ocorrencia_latitude',
         'ocorrencia_longitude',
         'investigacao_aeronave_liberada',
         'investigacao_status',
         'divulgacao_relatorio_numero',
         'divulgacao_relatorio_publicado',
         'divulgacao_dia_publicacao'], axis = 1, inplace = True) 

In [ ]:
df_cenipa.rename(columns={'ocorrencia_classificacao':'classificacao',
                   'ocorrencia_cidade':'cidade',
                   'ocorrencia_uf':'uf',
                   'ocorrencia_pais':'pais',
                   'ocorrencia_aerodromo':'aerodromo',
                   'ocorrencia_dia':'data',
                   'ocorrencia_hora':'horario',
                   'total_aeronaves_envolvidas':'n_aeronaves',
                   'ocorrencia_saida_pista':'saida_pista'}, inplace = True)

In [ ]:
df_cenipa.replace(['***','**NI'],pd.NA,inplace=True)

In [ ]:
df_cenipa.replace(pd.NA,np.NaN,inplace=True)

In [ ]:
df_cenipa

,codigo_ocorrencia,classificacao,cidade,uf,pais,aerodromo,data,horario,total_recomendacoes,n_aeronaves,saida_pista
0,81027,INCIDENTE GRAVE,SÃO PAULO,SP,BRASIL,SBSP,09/10/2022,16:30:00,0,1,SIM
1,81030,INCIDENTE,VITÓRIA,ES,BRASIL,SBVT,09/10/2022,16:20:00,0,1,NÃO
2,81023,INCIDENTE GRAVE,AMERICANA,SP,BRASIL,SDAI,06/10/2022,15:30:00,0,1,SIM
3,81029,INCIDENTE,BELO HORIZONTE,MG,BRASIL,SBBH,06/10/2022,00:44:00,0,1,NÃO
4,81025,INCIDENTE,MANAUS,AM,BRASIL,SBEG,05/10/2022,20:56:00,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...
6764,40324,INCIDENTE,PELOTAS,RS,BRASIL,SBPK,05/01/2010,19:25:00,0,1,NÃO
6765,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,BRASIL,NaN,04/01/2010,17:30:00,0,1,NÃO
6766,40211,INCIDENTE,RIO DE JANEIRO,RJ,BRASIL,NaN,03/01/2010,12:00:00,0,1,NÃO
6767,40349,INCIDENTE,BELÉM,PA,BRASIL,SBBE,03/01/2010,11:05:00,0,1,NÃO


In [ ]:
df_cenipa.isna().sum()

codigo_ocorrencia         0
classificacao             0
cidade                    0
uf                        4
pais                      0
aerodromo              2618
data                      0
horario                   2
total_recomendacoes       0
n_aeronaves               0
saida_pista               0
dtype: int64

In [ ]:
#Criar coluna periodo (data + horario)
df_cenipa['data'] = df_cenipa['data'].astype(str)
df_cenipa['periodo'] = pd.to_datetime(df_cenipa['data']+' '+df_cenipa['horario'])


In [ ]:
df_cenipa.drop(['periodo'],axis=1,inplace=True)

##Transformação Airbnb

In [ ]:
df_airbnb.rename(columns={'room_type':'tipo_quarto',
                   'room_shared':'compartilhado',
                   'room_private':'privado',
                   'person_capacity':'cap_max_pessoas',
                   'host_is_superhost':'superhost',
                   'biz':'comercial',
                   'cleanliness_rating':'avaliacao_limpeza',
                   'guest_satisfaction_overall':'satisfacao_usuarios',
                   'bedrooms':'numero_quartos','realSum':'valor_total',
                   'multi':'mult_quartos',
                   'dist':'dist_cidade',
                   'metro_dist':'dist_metro'}, inplace=True)

In [ ]:
df_airbnb

,Unnamed: 0,valor_total,tipo_quarto,compartilhado,privado,cap_max_pessoas,superhost,mult_quartos,comercial,avaliacao_limpeza,satisfacao_usuarios,numero_quartos,dist_cidade,dist_metro,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
0,0,194.033698,Private room,False,True,2.0,False,1,0,10.0,93.0,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772
1,1,344.245776,Private room,False,True,4.0,False,0,0,8.0,85.0,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432
2,2,264.101422,Private room,False,True,2.0,False,0,1,9.0,87.0,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103
3,3,433.529398,Private room,False,True,4.0,False,0,1,9.0,90.0,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663
4,4,485.552926,Private room,False,True,2.0,True,0,0,10.0,98.0,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,1098,2486.115342,Entire home/apt,False,False,2.0,False,0,0,10.0,100.0,1,2.375833,1.436054,181.198355,9.594573,225.304203,15.699521,4.85869,52.37677
1099,1099,233.637194,Private room,False,True,2.0,False,1,0,10.0,90.0,1,4.719736,0.322263,108.024880,5.719989,134.775277,9.391335,4.83611,52.34910
1100,1100,317.062311,Private room,False,True,2.0,False,1,0,10.0,92.0,1,0.598355,0.751993,440.476515,23.323524,625.947562,43.616927,4.88897,52.37798
1101,1101,1812.855904,Entire home/apt,False,False,4.0,False,0,0,8.0,84.0,5,1.943020,0.388532,257.948105,13.658524,336.589237,23.454022,4.90688,52.35794


In [ ]:
df_airbnb.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
df_airbnb.loc[df_airbnb.tipo_quarto == 'Private room',['tipo_quarto']] = 'QUARTO PRIVADO'
df_airbnb.loc[df_airbnb.tipo_quarto == 'Entire home/apt',['tipo_quarto']] = 'CASA/APTO COMPLETO'
df_airbnb.loc[df_airbnb.tipo_quarto == 'Shared room',['tipo_quarto']] = 'QUARTO COMPARTILHADO'

In [ ]:
df_airbnb

,valor_total,tipo_quarto,compartilhado,privado,cap_max_pessoas,superhost,mult_quartos,comercial,avaliacao_limpeza,satisfacao_usuarios,numero_quartos,dist_cidade,dist_metro,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
0,194.033698,QUARTO PRIVADO,False,True,2.0,False,1,0,10.0,93.0,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772
1,344.245776,QUARTO PRIVADO,False,True,4.0,False,0,0,8.0,85.0,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432
2,264.101422,QUARTO PRIVADO,False,True,2.0,False,0,1,9.0,87.0,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103
3,433.529398,QUARTO PRIVADO,False,True,4.0,False,0,1,9.0,90.0,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663
4,485.552926,QUARTO PRIVADO,False,True,2.0,True,0,0,10.0,98.0,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,2486.115342,CASA/APTO COMPLETO,False,False,2.0,False,0,0,10.0,100.0,1,2.375833,1.436054,181.198355,9.594573,225.304203,15.699521,4.85869,52.37677
1099,233.637194,QUARTO PRIVADO,False,True,2.0,False,1,0,10.0,90.0,1,4.719736,0.322263,108.024880,5.719989,134.775277,9.391335,4.83611,52.34910
1100,317.062311,QUARTO PRIVADO,False,True,2.0,False,1,0,10.0,92.0,1,0.598355,0.751993,440.476515,23.323524,625.947562,43.616927,4.88897,52.37798
1101,1812.855904,CASA/APTO COMPLETO,False,False,4.0,False,0,0,8.0,84.0,5,1.943020,0.388532,257.948105,13.658524,336.589237,23.454022,4.90688,52.35794


##Transformação Alespe

In [ ]:
df_alespe

,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
1,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,PAPER FACE COMERCIAL LTDA
2,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
...,...,...,...,...,...,...,...,...
542390,2007,300441,3,120.00,6306675000140.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",OPTEI EXPRESS LTDA.
542391,2007,300441,3,2096.80,52204617000122.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CHURRASCARIA JANDA LTDA.
542392,2007,300441,3,13.48,62925474000282.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",ASSOCIAÇÃO FUNC.ASSEMB.LEG. EST.SÃO PAULO
542393,2007,300441,3,38.08,66747577000170.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",IBIZA EUROPEIA ALIMENTOS LTDA


In [ ]:
df_alespe.dtypes

Ano             int64
Matricula       int64
Mes             int64
Valor         float64
CNPJ           object
Deputado       object
Tipo           object
Fornecedor     object
dtype: object

In [ ]:
df_alespe.loc[df_alespe.CNPJ == 'o',['CNPJ']] = 0
df_alespe.loc[df_alespe.CNPJ == 'O',['CNPJ']] = 0
df_alespe.loc[df_alespe.CNPJ == '.',['CNPJ']] = ''

##Enviando Tratados para o MongoDB

In [ ]:
db2 = client['Tratados']
collection_cenipa = db2['cenipa']
collection_airbnb = db2['airbnb']
collection_alespe = db2['alespe']

In [ ]:
df_dict_cenipa = df_cenipa.to_dict("records")
df_dict_airbnb = df_airbnb.to_dict("records")
df_dict_alespe = df_alespe.to_dict("recods")

collection_cenipa.insert_many(df_dict_cenipa)
collection_airbnb.insert_many(df_dict_airbnb)
collection_alespe.insert_many(df_dict_alespe)

<ipython-input-72-33967fb33c21>:3: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  df_dict_alespe = df_alespe.to_dict("recods")
